## Import

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2' # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!

import pandas as pd
import numpy as np
import torch
import transformers
import bitsandbytes as bnb



from transformers import AutoTokenizer, AdamW, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model,PeftModel
from tqdm import tqdm

#os.environ["TOKENIZERS_PARALLELISM"] = "true" 
#torch.backends.cuda.matmul.allow_tf32=True
#torch.set_float32_matmul_precision('medium')
#torch.backends.cudnn.benchmark = True

## Data Preprocessing

In [2]:
# 데이터 로드
model_id = 'beomi/SOLAR-KOEN-10.8B'
tokenizer = AutoTokenizer.from_pretrained(model_id,  eos_token='</s>')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # 패딩을 문장 뒤에 추가
max_length = 1024
dataset = load_dataset("jojo0217/korean_safe_conversation",split="train")


formatted_data = []
for row in dataset:
    input_text = f'''### User:\n{row['instruction']}\n\n### Assistant:\n{row['output']}'''
    input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length)
    formatted_data.append(input_ids)
print('Done.')


Done.


In [3]:
formatted_data = torch.cat(formatted_data, dim=0)

## Model Fine-tuning

In [ ]:
# 모델 로드



bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")


model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    #target_modules=["query_key_value"], 
    target_modules=[
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"],
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=formatted_data,
    args=transformers.TrainingArguments(
        num_train_epochs=2,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
      #  max_steps=50,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=1000,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False 
trainer.train()

Loading checkpoint shards:   0%|          | 0/23 [00:00<?, ?it/s]

trainable params: 31457280 || all params: 5645930496 || trainable%: 0.5571673264891711


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/leadawon5/decs_jupyter_lab/venvs311/llama2venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1000,1.732600
2000,1.697900
3000,1.693600
4000,1.705700
5000,1.693800
6000,1.703900
7000,1.698100
8000,1.684400
9000,1.682900
10000,1.672100


/home/leadawon5/decs_jupyter_lab/venvs311/llama2venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/leadawon5/decs_jupyter_lab/venvs311/llama2venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/

In [ ]:
new_model = "solar-ft"
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map="auto",
                                             #torch_dtype=torch.float32,
                                             )
model = PeftModel.from_pretrained(model, new_model,device_map="auto")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(new_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
formatted_data = []
qs = []
gts = []
for row in dataset:
    input_text = f'''### User:\n{row['instruction']}\n\n### Assistant:\n'''
    input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length)
    formatted_data.append(input_ids)
    qs.append(row['instruction'])
    gts.append(row['output'])
print('Done.')

In [ ]:
with open('bigdata/out.txt', 'w', encoding='utf-8') as file:
    pass


terminators = [
    tokenizer.eos_token_id,
    #tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

for idx in range(len(formatted_data)):
    
    print(f"{dix}/{len(formatted_data)}")
    with torch.no_grad():

        outputs = model.generate(
            formatted_data[idx],
            max_new_tokens=512,
            eos_token_id=terminators,
        #     do_sample=True,
        #     temperature=1,
        #     top_p=0.9,
        )
        full_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
        answer_start = full_text.find("\n\n### Assistant:") + len("\n\n### Assistant:")
        answer_only = full_text[answer_start:].strip()
        dic = {"question":qs[idx],"answer":answer_only,"goldanswer":gts[idx]}
        dic_str = '\n'.join([f"{key}: {value}" for key, value in dic.items()])
        with open('bigdata/out.txt', 'a', encoding='utf-8') as file:
            file.write(dic_str)